<a href="https://colab.research.google.com/github/wonjae124/Capston/blob/colab/Ongoing_widedeep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 참고 EDA : https://github.com/ksylvia16/Animal-Outcomes-Austin-TX/blob/main/code/02_Initial_Eda.ipynb
# 참고 코드 : https://github.com/CjMullins87/Animal-Shelter-Outcomes/blob/master/EDA.ipynb
# 트리,XGBoost 모델 사용 하는 사이트 : https://github.com/CjMullins87/Animal-Shelter-Outcomes/blob/master/Model%20Fitting%20and%20Review.ipynb
# 결과 비교 가능 사이트1 : https://github.com/CjMullins87/Animal-Shelter-Outcomes/blob/master/Executive%20Summary.pdf 다중 분류...
# https://jovian.ai/elissammi/austin-animal-center-eda-course-project/v/33?utm_source=embed#C68
# 동물의 이전에 입양된 횟수를 기록하는 데이터셋임. 단, 입양 여부만을 결정하는 이진분류로 만들어짐. https://github.com/ksylvia16/Animal-Outcomes-Austin-TX
!pip install pytorch-widedeep

     |████████████████████████████████| 21.1 MB 1.5 MB/s 
     |████████████████████████████████| 398 kB 38.7 MB/s 
     |████████████████████████████████| 1.4 MB 39.2 MB/s 
     |████████████████████████████████| 52.7 MB 78 kB/s 
     |████████████████████████████████| 134 kB 48.8 MB/s 
     |████████████████████████████████| 1.6 MB 45.1 MB/s 
     |████████████████████████████████| 225 kB 49.0 MB/s 
     |████████████████████████████████| 4.1 MB 43.0 MB/s 
     |████████████████████████████████| 125 kB 50.2 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [ ]:
pip install git+https://github.com/jrzaurin/pytorch-widedeep.git


  Cloning https://github.com/jrzaurin/pytorch-widedeep.git to /tmp/pip-req-build-s0uq03n4
  Running command git clone -q https://github.com/jrzaurin/pytorch-widedeep.git /tmp/pip-req-build-s0uq03n4


In [ ]:
!git clone https://github.com/jrzaurin/pytorch-widedeep
!cd pytorch-widedeep

Cloning into 'pytorch-widedeep'...
remote: Enumerating objects: 5305, done.
remote: Counting objects: 100% (3202/3202), done.
remote: Compressing objects: 100% (1983/1983), done.
remote: Total 5305 (delta 2387), reused 1922 (delta 1140), pack-reused 2103
Receiving objects: 100% (5305/5305), 86.77 MiB | 15.62 MiB/s, done.
Resolving deltas: 100% (3774/3774), done.


In [ ]:
!pip install -e .


ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content


In [2]:
import pandas as pd
import numpy as np
import torch
import time

from sklearn.model_selection import train_test_split

from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy
from pytorch_widedeep.datasets import load_adult



In [35]:
df = pd.read_csv('/content/drive/MyDrive/Wide_deep/aac_intakes_outcomes.csv')
df = df.fillna('?')

In [36]:
df["outcometype_label"] = (df["outcome_type"].apply(lambda x: 'Euthanasia' in x)).astype(int) # Return_to_Owner에 대한 이진분류 실행 예정



In [37]:
df_train, df_test = train_test_split(df, test_size = 0.2, stratify = df.outcometype_label)


In [38]:
df


,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,...,sex_upon_intake,age_upon_intake_(days),age_upon_intake_(years),intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,time_in_shelter_days,outcometype_label
0,Return_to_Owner,Neutered/Male,3650,10.000000,2017-12-07 14:07,12,2017,Dec-17,Thursday,0,...,Neutered/Male,3650,10.000000,2017-12-07 0:00,12,2017,Dec-17,Thursday,0.588194,0
1,Return_to_Owner,Neutered/Male,2555,7.000000,2014-12-20 16:35,12,2014,Dec-14,Saturday,16,...,Neutered/Male,2555,7.000000,2014-12-19 10:21,12,2014,Dec-14,Friday,1.259722,0
2,Return_to_Owner,Neutered/Male,2190,6.000000,2014-03-08 17:10,3,2014,Mar-14,Saturday,17,...,Neutered/Male,2190,6.000000,2014-03-07 14:26,3,2014,Mar-14,Friday,1.113889,0
3,Transfer,Neutered/Male,3650,10.000000,2014-04-07 15:12,4,2014,Apr-14,Monday,15,...,Neutered/Male,3650,10.000000,2014-04-02 15:55,4,2014,Apr-14,Wednesday,4.970139,0
4,Return_to_Owner,Neutered/Male,5840,16.000000,2013-11-16 11:54,11,2013,Nov-13,Saturday,11,...,Neutered/Male,5840,16.000000,2013-11-16 9:02,11,2013,Nov-13,Saturday,0.119444,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79667,Transfer,Unknown,14,0.038356,2018-03-29 18:14,3,2018,Mar-18,Thursday,18,...,Unknown,14,0.038356,2018-03-29 16:23,3,2018,Mar-18,Thursday,0.077083,0
79668,Euthanasia,Unknown,730,2.000000,2018-03-29 18:30,3,2018,Mar-18,Thursday,18,...,Unknown,730,2.000000,2018-03-29 17:13,3,2018,Mar-18,Thursday,0.053472,1
79669,Euthanasia,Unknown,365,1.000000,2018-03-29 18:28,3,2018,Mar-18,Thursday,18,...,Unknown,365,1.000000,2018-03-29 17:19,3,2018,Mar-18,Thursday,0.047917,1
79670,Return_to_Owner,Intact/Male,300,0.821918,2018-03-31 12:37,3,2018,Mar-18,Saturday,12,...,Intact/Male,150,0.410959,2018-03-29 18:19,3,2018,Mar-18,Thursday,1.762500,0


In [39]:
df.drop("outcome_type", axis = 1, inplace= True)
df.drop("outcome_monthyear", axis = 1, inplace=True)
df.drop("age_upon_outcome_(days)" ,axis = 1, inplace=True)
df.drop("outcome_year", axis = 1, inplace=True)


In [40]:
#df.drop("age_upon_outcome_(years)", inplace=True)
df.drop("outcome_weekday", axis = 1, inplace=True)
df.drop("outcome_hour", axis = 1, inplace=True)
df.drop("outcome_number", axis = 1, inplace=True)
df.drop("dob_year", axis = 1, inplace=True)
df.drop("dob_month", axis = 1, inplace=True)
df.drop("sex_upon_intake", axis = 1, inplace=True)
df.drop("intake_datetime", axis = 1, inplace=True)
df.drop("intake_month", axis = 1, inplace=True)
df.drop("intake_year", axis = 1, inplace=True)
df.drop("intake_monthyear", axis = 1, inplace=True)
df.drop("intake_weekday", axis = 1, inplace=True)
df.drop("dob_monthyear", axis = 1, inplace=True)



In [41]:
df.head()

,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,...,sex_upon_intake,age_upon_intake_(days),age_upon_intake_(years),intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,time_in_shelter_days,outcometype_label
0,Return_to_Owner,Neutered/Male,3650,10.0,2017-12-07 14:07,12,2017,Dec-17,Thursday,0,...,Neutered/Male,3650,10.0,2017-12-07 0:00,12,2017,Dec-17,Thursday,0.588194,0
1,Return_to_Owner,Neutered/Male,2555,7.0,2014-12-20 16:35,12,2014,Dec-14,Saturday,16,...,Neutered/Male,2555,7.0,2014-12-19 10:21,12,2014,Dec-14,Friday,1.259722,0
2,Return_to_Owner,Neutered/Male,2190,6.0,2014-03-08 17:10,3,2014,Mar-14,Saturday,17,...,Neutered/Male,2190,6.0,2014-03-07 14:26,3,2014,Mar-14,Friday,1.113889,0
3,Transfer,Neutered/Male,3650,10.0,2014-04-07 15:12,4,2014,Apr-14,Monday,15,...,Neutered/Male,3650,10.0,2014-04-02 15:55,4,2014,Apr-14,Wednesday,4.970139,0
4,Return_to_Owner,Neutered/Male,5840,16.0,2013-11-16 11:54,11,2013,Nov-13,Saturday,11,...,Neutered/Male,5840,16.0,2013-11-16 9:02,11,2013,Nov-13,Saturday,0.119444,0


In [42]:
wide_cols = [
             "breed",
             "animal_type",
             "color",
] #wide part는 memorization 담당(고정요소. 타고 태어난 요소)

In [43]:
crossed_cols = [("intake_type","breed"),("color","breed")] # wide_cols에 있는 녀석들로 가능해보임. 여기에 비선형 데이터를 추가 할 수 있다고

cat_embed_cols = [
             "breed",
             "animal_type",
             "color",
             "intake_type",
             "intake_condition",
             "sex_upon_intake",
             "sex_upon_outcome",
]  # deep part는 Generalization 담당(고정요소 + 변동 요소)

continuous_cols = ["age_upon_intake_(years)","age_upon_outcome_(years)","time_in_shelter_days"] # 

target = "outcometype_label" 
target = df_train[target].values

In [44]:
wide_preprocessor = WidePreprocessor(wide_cols = wide_cols, crossed_cols = crossed_cols) #????
X_wide = wide_preprocessor.fit_transform(df_train) # 왜이렇게 많이 뭔가 전처리 하지?

tab_preprocessor = TabPreprocessor(
    cat_embed_cols = cat_embed_cols, continuous_cols= continuous_cols
)

X_tab = tab_preprocessor.fit_transform(df_train)

In [45]:
wide = Wide(input_dim=np.unique(X_wide).shape[0], pred_dim = 1)
tab_mlp = TabMlp(
    column_idx = tab_preprocessor.column_idx,
    cat_embed_input = tab_preprocessor.cat_embed_input,
    continuous_cols = continuous_cols,
)

model = WideDeep(wide = wide, deeptabular = tab_mlp)

In [46]:
trainer = Trainer(model, objective = "binary", metrics = [Accuracy])
# 앗! 여기서 obejctive 말고 다른 것을 할 수 있겠다.
# 그런데, 검증 함수 그런건 어딨냐?.. 아까 있었는데

start = time.time()


trainer.fit(
    X_wide=X_wide,
    X_tab = X_tab,
    target = target,
    n_epochs = 50,
    batch_size = 256,
)

end = time.time()
total_time = (end-start)/60

print("excution_time : {:.1f}min".format(total_time))

epoch 1:  34%|███▍      | 85/249 [00:02<00:03, 43.22it/s, loss=0.226, metrics={'acc': 0.9306}]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fcb3e7f59e0>
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
Traceback (most recent call last):
    self._shutdown_workers()
epoch 1:  34%|███▍      | 85/249 [00:02<00:03, 43.22it/s, loss=0.226, metrics={'acc': 0.9306}]  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
epoch 1:  34%|███▍      | 85/249 [00:02<00:03, 43.22it/s, loss=0.225, metrics={'a

KeyboardInterrupt: ignored

In [ ]:
df.outcome_type.value_counts()


In [ ]:
X_wide_te = wide_preprocessor.transform(df_test) #wide랑 tab이랑 무슨 차이지?
X_tab_te = tab_preprocessor.transform(df_test)
preds = trainer.predict(X_wide = X_wide_te, X_tab = X_tab_te) #한쪽은 정답이고 한쪽은 입력 아닌가?

In [ ]:
#trainer와 torch는 무슨 차이지?
# Option 1: this will also save training history and lr history if the

trainer.save(path="model_weights", save_state_dict = True)

# Option 2: save as any other torch model

torch.save(model.state_dict(), "model_weights/wd_model.pt") # 이게 도대체 뭐야? 허허... 왜 파일 확장자가 pt인거지?

model_new = WideDeep(wide = wide, deeptabular = tab_mlp) # wide랑 tab_mlp랑 왜 따로 트랜스폼 해두었을까?

model_new.load_state_dict(torch.load("model_weights/wd_model.pt")) 

trainer_new = Trainer(model_new, objective = 'binary') #trainer는 언제 만들어서 언제 활용했지?

preds = trainer_new.predict(X_wide=X_wide, X_tab = X_tab)

In [ ]:
!pytest tests